In [1]:
import json 
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, concatenate, Dropout, GlobalAveragePooling1D
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification, AutoTokenizer, AutoModel, AutoModelForMaskedLM

import numpy as np
import cv2
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from keras.saving import register_keras_serializable
import time

In [2]:
!pip install einops 

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [3]:
!pip install git+https://github.com/Xuanhoang214/VietnameseTextNormalizer.git

  Cloning https://github.com/Xuanhoang214/VietnameseTextNormalizer.git to /tmp/pip-req-build-qtlnr4mg
  Running command git clone --filter=blob:none --quiet https://github.com/Xuanhoang214/VietnameseTextNormalizer.git /tmp/pip-req-build-qtlnr4mg
  Resolved https://github.com/Xuanhoang214/VietnameseTextNormalizer.git to commit d0ec5e0c046fef16e03b0bfb4bc7a72dbfe38eeb
  Preparing metadata (setup.py) ... done
  Created wheel for VietnameseTextNormalizer: filename=VietnameseTextNormalizer-1.2a0-cp310-cp310-linux_x86_64.whl size=13548147 sha256=d3924ea66512a6d1901b4639c375b362f6e0765038166d16e6ab1bc298172003
  Stored in directory: /tmp/pip-ephem-wheel-cache-zwjq089c/wheels/f3/a3/8b/01c922997e3fd6caae860b93f268b6de3a9c59b10b6ee7acad
Successfully built VietnameseTextNormalizer


In [4]:
from PyVietnameseTextNormalizer import Normalizer
N = Normalizer()

In [5]:
def read_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
        data_list = [{**value} for key, value in data.items()]
        
        df = pd.DataFrame(data_list)
        
    return df

In [6]:
path = '/kaggle/input/data-ai002/data_AI002/data_AI002/'

In [7]:
json_path = '/kaggle/input/data-ai002/data-AI002.json'

In [8]:
df = read_json(json_path)
df

,image,caption,label
0,8ae451edcd8ebf697f8763ece249115813149c55733bf8...,Cô ấy trên mạng vs cô ấy ngoài đời =))),sarcasm
1,35370ffd6c791d6f8c4ab3dd4363ed468fab41e4824ee9...,Người tâm linh giao tiếp với người thực tế :))),sarcasm
2,316fdd1477725b9fb1a55015ac06b68b92b50bd4303e08...,Hình như Trăng hôm nay đẹp quá mọi người ạ! 😃 ...,sarcasm
3,8a0f34e0e30e4e5cfb306933c1d25fa801a5da78646b59...,MỌI NGƯỜI NGHĨ SAO VỀ PHÁT BIỂU CỦA SHARK VIỆT...,not-sarcasm
4,e517a5e95d1065886a7c815e82fe254381d4f9f4b244d4...,2 tay hai nàng chứ việc gì phải lệ hai hàng,sarcasm
...,...,...,...
11902,3c643826258f8aacc8a98d8e24956f909797010f1e80bd...,Mẹ biết mẹ buồn ó 🐧,sarcasm
11903,09f5adf3e555d3066eae0be356e5ce797c066706e7c808...,Ủa ăn đi anh sao tự nhiên rén vậy? \n#KFCViet...,sarcasm
11904,b11515c1aa521da4f6d0cd6464e5ea3e030662cdd43f7f...,Mẹ không phát hiện sớm là vài tháng nữa con Ch...,sarcasm
11905,8a83634808704a5c7493327893f793effd6e78cb037ac4...,Anh em h.út ch.ích gánh còng lưng :(((,sarcasm


In [9]:
class CombinedSarcasmClassifier:
    def __init__(self):
        self.model = None
        self.vit_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
        self.vit_model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")
        self.jina_tokenizer = AutoTokenizer.from_pretrained("uitnlp/visobert")
        self.jina_model = AutoModel.from_pretrained("uitnlp/visobert", 
                                                   trust_remote_code=True,
                                                   torch_dtype=torch.float32)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Define label mapping
        self.label_mapping = {
            'not-sarcasm': 0,
            'sarcasm': 1
        }
        
        # Ensure models are in float32
        self.vit_model.to(self.device).to(torch.float32)
        self.jina_model.to(self.device).to(torch.float32)

    def encode_labels(self, labels):
        """Convert text labels to one-hot encoded format"""
        numerical_labels = [self.label_mapping[label] for label in labels]
        return tf.keras.utils.to_categorical(numerical_labels, num_classes=2)

    def decode_labels(self, one_hot_labels):
        numerical_labels = np.argmax(one_hot_labels, axis=1)
        reverse_mapping = {v: k for k, v in self.label_mapping.items()}
        return [reverse_mapping[idx] for idx in numerical_labels]

    def build(self, image_dim=1768, text_dim=768):
        image_input = Input(shape=(image_dim,), name='image_input')
        text_input = Input(shape=(text_dim,), name='text_input')

        # Image processing branch
        image_dense = Dense(1024, activation='relu')(image_input)
        image_dropout = Dropout(0.3)(image_dense)
        image_dense2 = Dense(756, activation='relu')(image_dropout)

        # Text processing branch
        text_dense = Dense(512, activation='relu')(text_input)
        text_dropout = Dropout(0.3)(text_dense)
        text_dense2 = Dense(256, activation='relu')(text_dropout)

        # Combine both branches
        combined = concatenate([image_dense2, text_dense2])
        dense_combined = Dense(512, activation='relu')(combined)
        dropout_combined = Dropout(0.3)(dense_combined)
        dense_combined2 = Dense(256, activation='relu')(dropout_combined)
        dropout_combined2 = Dropout(0.3)(dense_combined2)
        output = Dense(2, activation='softmax', name='output')(dropout_combined2)

        self.model = Model(inputs=[image_input, text_input], outputs=output)

    def preprocess_data(self, images, texts, ocr_file_path, is_test=0):
        
        # Read OCR data
        ocr_df = pd.read_csv(ocr_file_path)
        ocr_df.columns = ['image_name', 'ocr_text']  # Rename columns
        ocr_dict = dict(zip(ocr_df.image_name, ocr_df.ocr_text))
        
        combined_features = []
        total_items = len(images)
        
        print("\nProcessing images and texts:")
        for i, image in enumerate(images, 1):
            print(f"Processing item {i}/{total_items}: {image}", end='\r')
            
            # Process image
            temp = cv2.imread(path + image)
            inputs = self.vit_processor(images=temp, return_tensors="pt").to(self.device)
            with torch.no_grad():
                outputs = self.vit_model(**inputs)
            img_features = outputs.logits.cpu().numpy().squeeze()
            try:
            # Get corresponding OCR text
                ocr_text = ocr_dict.get(image, '')  # Get OCR text or empty string if not found
                ocr_text = N.normalize(ocr_text)
                # Combine original text with OCR text
                combined_text = f"{ocr_text}".strip()
                
                # Process combined text with JINA model
                text_inputs = self.jina_tokenizer(
                    combined_text, 
                    return_tensors="pt", 
                    padding=True, 
                    truncation=True, 
                    max_length=512
                ).to(self.device)
                
                with torch.no_grad():
                    text_outputs = self.jina_model(**text_inputs)
                text_feats = text_outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
            except Exception as e:
                print(f"\nError processing item {i}: {str(image)}")
                text_feats = np.zeros(768)
            # Concatenate image and text features
            combined_feature = np.concatenate([img_features, text_feats])
            combined_features.append(combined_feature)
            
            
        print("\nProcessing texts:")
        text_features = []
        total_texts = len(texts)
        for i, text in enumerate(texts, 1):
            try:
                print(f"Processing text {i}/{total_texts}", end='\r')
                inputs = self.jina_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
                
                with torch.no_grad():
                    outputs = self.jina_model(**inputs)
                features = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
                text_features.append(features)
            except Exception as e:
                print(f"\nError processing text: {str(e)}")
                text_features.append(np.zeros(768))  # Handle errors by adding zero vectors

        print("\nPreprocessing completed!")
        return np.array(combined_features), np.array(text_features)


    @staticmethod
    @register_keras_serializable(package="Custom", name="f1_macro")
    def f1_macro(y_true, y_pred):
        """Custom F1 macro metric for Keras"""
        y_true_class = tf.argmax(y_true, axis=1)
        y_pred_class = tf.argmax(y_pred, axis=1)
        
        f1_scores = []
        for i in range(2):  # Update this based on the number of classes
            true_positives = tf.reduce_sum(tf.cast(
                tf.logical_and(tf.equal(y_true_class, i), tf.equal(y_pred_class, i)),
                tf.float32
            ))
            false_positives = tf.reduce_sum(tf.cast(
                tf.logical_and(tf.not_equal(y_true_class, i), tf.equal(y_pred_class, i)),
                tf.float32
            ))
            false_negatives = tf.reduce_sum(tf.cast(
                tf.logical_and(tf.equal(y_true_class, i), tf.not_equal(y_pred_class, i)),
                tf.float32
            ))
            
            precision = true_positives / (true_positives + false_positives + tf.keras.backend.epsilon())
            recall = true_positives / (true_positives + false_negatives + tf.keras.backend.epsilon())
            f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
            f1_scores.append(f1)
        
        return tf.reduce_mean(f1_scores)


    def learning_rate_schedule(self, epoch, lr):
        """Learning rate scheduler function."""
        if 5 <= epoch :
            return lr * 0.5
        elif 20 <= epoch < 21:
            return lr * 0.01
        elif 21 <= epoch :
            return lr * 0.005
        return lr

    def train(self, x_train_images, x_train_texts, y_train):
        print("Starting preprocessing...")
        image_features, text_features = self.preprocess_data(x_train_images, x_train_texts,ocr_file_path='/kaggle/input/csv-ocr/ocr_text_easyocr.csv')

        print(f"Image feature shape: {image_features.shape}")
        print(f"Text feature shape: {text_features.shape}")
        
        # Convert labels to numerical format for stratification
        numerical_labels = [self.label_mapping[label] for label in y_train]
        
        # Perform stratified split
        sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
        train_idx, val_idx = next(sss.split(image_features, numerical_labels))
        
        # Split the data
        train_image_features = image_features[train_idx]
        train_text_features = text_features[train_idx]
        val_image_features = image_features[val_idx]
        val_text_features = text_features[val_idx]
        
        # Encode labels after splitting
        y_train = np.array(y_train)  
        y_train_encoded = self.encode_labels(y_train[train_idx])
        y_val_encoded = self.encode_labels(y_train[val_idx])
        
        initial_lr = 7.5e-5

        print("\nCompiling model...")
        self.model.compile(
            optimizer=tf.keras.optimizers.AdamW(learning_rate=initial_lr),
            loss='categorical_crossentropy',
            metrics=[tf.keras.metrics.AUC(), CombinedSarcasmClassifier.f1_macro]
        )
        
        class BatchProgressCallback(tf.keras.callbacks.Callback):
            def on_epoch_begin(self, epoch, logs=None):
                print(f"\nEpoch {epoch + 1} starting...")
            
            def on_batch_begin(self, batch, logs=None):
                print(f"Training batch {batch + 1}", end='\r')

        lr_scheduler = LearningRateScheduler(self.learning_rate_schedule)

        print("\nStarting training...")
        history = self.model.fit(
            [train_image_features, train_text_features],
            y_train_encoded,
            epochs=25,
            batch_size=16,
            validation_data=([val_image_features, val_text_features], y_val_encoded),
            callbacks=[BatchProgressCallback(), lr_scheduler]
        )
        
        print("\nTraining completed!")
        return history

    def predict(self, x_test_images, x_test_texts):
        print("Preprocessing test data...")
        # Ghi lại thời điểm bắt đầu
        start_preprocessing = time.time()
        
        image_features, text_features = self.preprocess_data(
            x_test_images, 
            x_test_texts, 
            ocr_file_path='/kaggle/input/csv-ocr/ocr_text_easyocr.csv', 
            is_test=1
        )
        
        # Đo thời gian cho giai đoạn tiền xử lý
        end_preprocessing = time.time()
        preprocessing_time = end_preprocessing - start_preprocessing
        print(f"Preprocessing completed in {preprocessing_time:.2f} seconds.")

        print("Making predictions...")
        # Ghi lại thời điểm bắt đầu dự đoán
        start_prediction = time.time()

        # Thực hiện dự đoán
        predictions = self.model.predict([image_features, text_features])
        
        # Đo thời gian cho giai đoạn dự đoán
        end_prediction = time.time()
        prediction_time = end_prediction - start_prediction
        
        # Tính thời gian trung bình mỗi mẫu
        num_samples = len(x_test_images)
        average_prediction_time = (preprocessing_time + prediction_time) / num_samples

        print(f"Prediction completed in {prediction_time:.2f} seconds.")
        print(f"Average prediction time per sample: {average_prediction_time:.4f} seconds.")

        # Giải mã nhãn dự đoán
        return self.decode_labels(predictions)

    def load(self, model_file):
        self.model = load_model(model_file, custom_objects={'f1_macro': self.f1_macro})

    def save(self, model_file):
        self.model.save(model_file)

    def summary(self):
        self.model.summary()


In [10]:
x_images = df['image']
x_captions = df['caption']
y = df['label']

In [11]:
classifier = CombinedSarcasmClassifier()
classifier.build()

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/471k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/390M [00:00<?, ?B/s]

Some weights of XLMRobertaModel were not initialized from the model checkpoint at uitnlp/visobert and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
x_train_images, x_test_images, x_train_captions, x_test_captions, y_train, y_test = train_test_split(
    x_images, x_captions, y, test_size=0.2, random_state=42, stratify=y
)

In [13]:
classifier.train(x_train_images, x_train_captions, y_train)

Starting preprocessing...

Processing images and texts:
Processing item 6/9525: 937fa5ec65c59c0e2daa6dea25277b2b99aaec2d0b8bb6ae2a8f9d981d68b825.jpg
Error processing item 6: 937fa5ec65c59c0e2daa6dea25277b2b99aaec2d0b8bb6ae2a8f9d981d68b825.jpg
Processing item 13/9525: 9089f19fc56e680bf45f34cfb4aec8661f2cd79c5d316aead2f32fb1f37d6c1e.jpg
Error processing item 13: 9089f19fc56e680bf45f34cfb4aec8661f2cd79c5d316aead2f32fb1f37d6c1e.jpg
Processing item 17/9525: 386a067f5cbef24595a5c93f7d8902bf10a265cae746fcd79eee21c15a083394.jpg
Error processing item 17: 386a067f5cbef24595a5c93f7d8902bf10a265cae746fcd79eee21c15a083394.jpg
Processing item 18/9525: a5058c8c9c2e19e0dd95b5ba96b138495c8aaf9e498ea52a95e2eb138c556ee9.jpg
Error processing item 18: a5058c8c9c2e19e0dd95b5ba96b138495c8aaf9e498ea52a95e2eb138c556ee9.jpg
Processing item 20/9525: e44a36a9e810a42415c33f3518835e327728b009356d27a7ad9d8876990dc9e8.jpg
Error processing item 20: e44a36a9e810a42415c33f3518835e327728b009356d27a7ad9d8876990dc9e8.jpg
P

In [14]:
classifier.save('model.keras')

In [15]:
classifier.load('model.keras')

In [16]:
predictions = classifier.predict(x_test_images, x_test_captions)

Preprocessing test data...

Processing images and texts:
Processing item 2/2382: a3074837a14e41037011ea894599eaa19a5c37bdceca8e8926a614b0981bcd75.jpg
Error processing item 2: a3074837a14e41037011ea894599eaa19a5c37bdceca8e8926a614b0981bcd75.jpg
Processing item 7/2382: c7e134dfaa45c9d9d43e7b3845f547c83e8f061091cda3f0511212778ed06c56.jpg
Error processing item 7: c7e134dfaa45c9d9d43e7b3845f547c83e8f061091cda3f0511212778ed06c56.jpg
Processing item 9/2382: 502415796bda4215db3a8cce9d9b409f88f858a6d8c0179049d35ab8d7d0ae28.jpg
Error processing item 9: 502415796bda4215db3a8cce9d9b409f88f858a6d8c0179049d35ab8d7d0ae28.jpg
Processing item 15/2382: cbeef4a861a5e65590946a7121b857218488cb55b49e3f76fbfff8e587e04294.jpg
Error processing item 15: cbeef4a861a5e65590946a7121b857218488cb55b49e3f76fbfff8e587e04294.jpg
Processing item 19/2382: d629644490ce5b84e036242565dbbca22cf86811457fb2be74902e124f4b5547.jpg
Error processing item 19: d629644490ce5b84e036242565dbbca22cf86811457fb2be74902e124f4b5547.jpg
Proc

In [17]:
from sklearn.metrics import classification_report

In [18]:
report = classification_report(predictions, y_test, target_names=['not-sarcasm','sarcasm'])
print("\nClassification Report:\n", report)



Classification Report:
               precision    recall  f1-score   support

 not-sarcasm       0.70      0.78      0.74      1071
     sarcasm       0.80      0.72      0.76      1311

    accuracy                           0.75      2382
   macro avg       0.75      0.75      0.75      2382
weighted avg       0.76      0.75      0.75      2382

